In [1]:
#!pip install georasters
#!pip install geoindex
#!pip install spark
#!pip install pyspark

In [2]:
import pandas as pd
import georasters as gr
import matplotlib.pyplot as plt
from geoindex import GeoGridIndex, GeoPoint
import os
from tqdm import tqdm, tqdm_pandas

In [3]:
pd.set_option('display.max_columns', 100)

## Load Water Data

In [4]:
os.getcwd()

'/Users/alexjamar/Documents/Berkeley/Capstone/Capstone_Water_Access/Population'

In [5]:
water_data = pd.read_csv("/Users/alexjamar/Documents/Berkeley/Capstone/Capstone_Water_Access//data/Water_Point_Data_Exchange_Complete_Dataset.csv")

/Users/alexjamar/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2,5,7,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
water_data.head()

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality
0,1,"Tanzania, United Republic of",Spring,Gravity Communal standpipe,no,NaN,Never pay,RWE,1972.0,Status:Not functional|Quantity:Dry|Quality:Soft,SNV,Arusha,Longido,wpdx-00000001,01/05/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.83257,36.3090,"(-2.83257, 36.309)",1,NaN,NaN,NaN
1,2,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Breakdown Year:2007|Reason N...,SNV,Arusha,Longido,wpdx-00000002,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.84781,36.5284,"(-2.84781, 36.5284)",1,NaN,NaN,NaN
2,3,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,SNV,Arusha,Longido,wpdx-00000003,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.84796,36.5288,"(-2.84796, 36.5288)",1,NaN,NaN,NaN
3,4,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,SNV,Arusha,Longido,wpdx-00000004,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.85318,36.5276,"(-2.85318, 36.5276)",1,NaN,NaN,NaN
4,5,Nicaragua,NaN,NaN,yes,NaN,NaN,NaN,2011.0,NaN,El Porvenier,Matagalpa,Terrabona,wpdx-00000005,01/01/2011 12:00:00 AM +0000,NI,NaN,http://www.dropbox.com/s/0f70mln5roxhy29/ProjT...,https://www.waterpointdata.org/dataset/resourc...,NaN,"#data_lnk, #lon_deg, #lat_deg, #country_id",04/08/2016 08:35:49 AM +0000,04/08/2016 08:35:49 AM +0000,12.74070,-86.0121,"(12.7407, -86.0121)",1,NaN,NaN,NaN


In [7]:
liberia_water = water_data[water_data['#country_name'] == 'Liberia']

In [8]:
liberia_water.head()

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality
103493,162435,Liberia,Manual pump on hand-dug well,Afridev,no,Community Management,Yes at least once a month,icrc - Red Cross (ICRC/LNRC),2007.0,Broken Down System. Pump stolen,WASH Liberia,Lofa,Voinjama,wpdx-00162435,01/01/2010 12:00:00 AM +0000,LR,1,http://www.dropbox.com/s/gra1jff02jbrgpo/Publi...,NaN,http://flowliberia.s3.amazonaws.com/images/wfp...,"#country_id, #management, #status, #status_id",10/26/2015 11:31:35 PM +0000,10/26/2015 11:31:35 PM +0000,8.36505,-9.79305,"(8.36505, -9.79305)",1,NaN,NaN,NaN
103494,162436,Liberia,Manual pump on hand-dug well,Afridev,yes,Community Management,Yes at least once a month,icrc - Red Cross (ICRC/LNRC),2010.0,Working but with problems. Well damaged,WASH Liberia,Lofa,Voinjama,wpdx-00162436,01/01/2010 12:00:00 AM +0000,LR,1,http://www.dropbox.com/s/gra1jff02jbrgpo/Publi...,NaN,http://flowliberia.s3.amazonaws.com/images/wfp...,"#country_id, #management, #status, #status_id",10/26/2015 11:31:35 PM +0000,10/26/2015 11:31:35 PM +0000,8.36514,-9.78684,"(8.36514, -9.78684)",1,NaN,NaN,NaN
103495,162437,Liberia,Manual pump on hand-dug well,Afridev,no,Community Management,Yes but only in case of breakdown,Oxfam - Oxfam,1995.0,Broken Down System. Pump broken,WASH Liberia,Grand Cape Mount,Porkpa,wpdx-00162437,02/01/2011 12:00:00 AM +0000,LR,0,http://www.dropbox.com/s/gra1jff02jbrgpo/Publi...,NaN,http://flowliberia.s3.amazonaws.com/images/wfp...,"#country_id, #management, #status, #status_id",10/26/2015 11:31:35 PM +0000,10/26/2015 11:31:35 PM +0000,7.15139,-11.19280,"(7.15139, -11.1928)",1,NaN,NaN,NaN
103496,162438,Liberia,Manual pump on hand-dug well,Afridev,yes,Community Management,Yes but only in case of breakdown,WVL - Other NGO/Company,2008.0,Working and protected. Not damaged,WASH Liberia,Grand Cape Mount,Porkpa,wpdx-00162438,02/01/2011 12:00:00 AM +0000,LR,0,http://www.dropbox.com/s/gra1jff02jbrgpo/Publi...,NaN,http://flowliberia.s3.amazonaws.com/images/wfp...,"#country_id, #management, #status, #status_id",10/26/2015 11:31:35 PM +0000,10/26/2015 11:31:35 PM +0000,7.15072,-11.19250,"(7.15072, -11.1925)",1,NaN,NaN,NaN
103497,162439,Liberia,Manual pump on borehole,India Mark,no,Community Management,Yes but only in case of breakdown,MRD - Other NGO/Company,NaN,Broken Down System. Apron damaged,WASH Liberia,Grand Cape Mount,Porkpa,wpdx-00162439,02/01/2011 12:00:00 AM +0000,LR,0,http://www.dropbox.com/s/gra1jff02jbrgpo/Publi...,NaN,http://flowliberia.s3.amazonaws.com/images/wfp...,"#country_id, #management, #status, #status_id",10/26/2015 11:31:35 PM +0000,10/26/2015 11:31:35 PM +0000,7.15209,-11.19240,"(7.15209, -11.1924)",1,NaN,NaN,NaN


## Radius-based Search

In [9]:
liberia_pop = pd.read_csv("Liberia_pop_data.csv")

In [10]:
#Create Geo Index of Liberia Population Data
geo_index = GeoGridIndex()
for index, row in liberia_pop.iterrows():
    geo_index.add_point(GeoPoint(row['y'], row['x'], ref=row['value']))

In [11]:
#Create Geo Index of Liberia Water Point Data
geo_index_water = GeoGridIndex()
for index, row in liberia_water.iterrows():
    try:
        geo_index_water.add_point(GeoPoint(row['#lat_deg'], row['#lon_deg'], ref=row['#status_id']))
    except:
        print("Row ID:", row['Row ID'])
        print("Lat:", row['#lat_deg'])
        print("Long:", row['#lon_deg'])

In [12]:
#Calculate population within x distance of water point
def calculate_population_within_x_km(row, index, km):
    center_point = GeoPoint(row['#lat_deg'], row['#lon_deg'])
    total_population = 0
    try:
        for point,distance in index.get_nearest_points(center_point, km, 'km'):
            total_population += point.ref
    except:
        print("Invalid data - Record skipped")
    #print("Total population within", km, "kilometers:", int(total_population))
    return total_population

In [13]:
#Calculate # of water points within x distance of water point
def calculate_water_points_within_x_km(row, index, km):
    center_point = GeoPoint(row['#lat_deg'], row['#lon_deg'])
    total_water_points = 0
    total_functioning_water_points = 0
    try:
        for point,distance in index.get_nearest_points(center_point, km, 'km'):
            total_water_points += 1
            if point.ref == 'yes':
                total_functioning_water_points += 1
    except:
        print("Invalid data - Record skipped")
        
    #Adjust for self
    total_water_points -= 1
    if row['#status_id'] == 'yes':
        total_functioning_water_points -= 1
    return (total_water_points, total_functioning_water_points)

In [14]:
#Add 1 km radius population data to each well
tqdm.pandas()
liberia_water['1 km population'] = liberia_water.progress_apply(lambda row: calculate_population_within_x_km(row, geo_index, 1),
                                                  axis = 1)

100%|██████████| 14053/14053 [14:27:05<00:00,  3.48s/it]     
/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [15]:
#Add 1 km radius water point data to each water point
tqdm.pandas()
liberia_water['1 km water points (total, functioning)'] = liberia_water.progress_apply(lambda row: calculate_water_points_within_x_km(row, geo_index_water, 1),
                                                  axis = 1)

100%|██████████| 14053/14053 [02:57<00:00, 79.27it/s] 
/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [16]:
liberia_water.head(2)

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality,1 km population,"1 km water points (total, functioning)"
103493,162435,Liberia,Manual pump on hand-dug well,Afridev,no,Community Management,Yes at least once a month,icrc - Red Cross (ICRC/LNRC),2007.0,Broken Down System. Pump stolen,WASH Liberia,Lofa,Voinjama,wpdx-00162435,01/01/2010 12:00:00 AM +0000,LR,1,http://www.dropbox.com/s/gra1jff02jbrgpo/Publi...,NaN,http://flowliberia.s3.amazonaws.com/images/wfp...,"#country_id, #management, #status, #status_id",10/26/2015 11:31:35 PM +0000,10/26/2015 11:31:35 PM +0000,8.36505,-9.79305,"(8.36505, -9.79305)",1,NaN,NaN,NaN,277.462,"(1, 1)"
103494,162436,Liberia,Manual pump on hand-dug well,Afridev,yes,Community Management,Yes at least once a month,icrc - Red Cross (ICRC/LNRC),2010.0,Working but with problems. Well damaged,WASH Liberia,Lofa,Voinjama,wpdx-00162436,01/01/2010 12:00:00 AM +0000,LR,1,http://www.dropbox.com/s/gra1jff02jbrgpo/Publi...,NaN,http://flowliberia.s3.amazonaws.com/images/wfp...,"#country_id, #management, #status, #status_id",10/26/2015 11:31:35 PM +0000,10/26/2015 11:31:35 PM +0000,8.36514,-9.78684,"(8.36514, -9.78684)",1,NaN,NaN,NaN,230.492,"(1, 0)"


In [17]:
def clean_water_data(df):
    df['1 km population'] = df['1 km population'].apply(lambda x: int(round(x,0)))
    df['1 km total water points'] = df['1 km water points (total, functioning)'].apply(lambda x: str(x).split(',')[0][1:])
    df['1 km functioning water points'] = df['1 km water points (total, functioning)'].apply(lambda x: str(x).split(',')[1][:-1])
    df = df[['Row ID', '#country_name', '#status_id', '#lat_deg', '#lon_deg', '1 km population',
                        '1 km total water points', '1 km functioning water points']]
    
    return df

In [18]:
liberia_abridged = clean_water_data(liberia_water)

/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [19]:
liberia_abridged.head()

,Row ID,#country_name,#status_id,#lat_deg,#lon_deg,1 km population,1 km total water points,1 km functioning water points
103493,162435,Liberia,no,8.36505,-9.79305,277,1,1
103494,162436,Liberia,yes,8.36514,-9.78684,230,1,0
103495,162437,Liberia,no,7.15139,-11.19280,277,2,1
103496,162438,Liberia,yes,7.15072,-11.19250,278,2,0
103497,162439,Liberia,no,7.15209,-11.19240,275,2,1


In [20]:
liberia_abridged.to_csv("Abridged_Liberia_Water_and_Population_Data.csv", index = False)